In [3]:
import cv2
import numpy as np
import math
import scipy.fftpack
from anytree import Node, RenderTree, PreOrderIter, AsciiStyle

In [4]:
img = cv2.imread("kodim14.png")

ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

In [5]:
def encode(ycbcr):
    y, cb, cr = cv2.split(ycbcr)
    dim = (int(ycbcr.shape[1]/2), int(ycbcr.shape[0]/2))
    
    resizedCb = cv2.resize(cb, dim)
    resizedCr = cv2.resize(cr, dim)
    

    return y, resizedCb, resizedCr

def decode(y, halfcb, halfcr):
    dim = (int(halfcb.shape[1]*2), int(halfcb.shape[0]*2))
    
    halfcb = np.array(halfcb, dtype='uint8')
    halfcr = np.array(halfcr, dtype='uint8')
    y = np.array(y, dtype='uint8')
    
    originalCb = cv2.resize(halfcb, dim)
    originalCr = cv2.resize(halfcr, dim)

    merged = cv2.merge((y, originalCb, originalCr))
    
    merged = cv2.cvtColor(merged, cv2.COLOR_YCrCb2BGR)
    
    cv2.imwrite('01.png',merged)
    return merged


In [6]:
def divide8(y, cb, cr):
    h, w = y.shape
        
    YBlocks = []
    CbBlocks = []
    CrBlocks = []
    
    
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            YBlocks.append(y[row : row + 8, col : col + 8])
    
    h, w = cb.shape
            
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            CbBlocks.append(cb[row : row + 8, col : col + 8])
            
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            CrBlocks.append(cr[row : row + 8, col : col + 8])
    
    return YBlocks, CbBlocks, CrBlocks

def undivide8(YBlocks, CbBlocks, CrBlocks, w, h):
    
    for k in range(len(YBlocks)):
        i = k // (w / 8) # quel bloc (ligne)
        j = k % (w / 8) # quel bloc (colonne)
        block = YBlocks[k] 

        for n in range(8): # quel pixel dans le bloc (ligne)
            for m in range(8): #quel pixel dans le bloc (colonne
                y[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    for k in range(len(CbBlocks)):
        i = k // (w / 16)
        j = k % (w / 16)
        block = CbBlocks[k]
        for n in range(8):
            for m in range(8):
                cb[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    for k in range(len(CrBlocks)):
        i = k // (w / 16)
        j = k % (w / 16)
        block = CrBlocks[k]
        for n in range(8):
            for m in range(8):
                cr[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    return y, cb, cr

In [7]:
def dctTransform(matrix):
    m = 8
    n = 8
    dct = [[0 for x in range(m)] for y in range(n)] 

    
    for ligne in range(m):
        for col in range(n):
            if (ligne == 0):
                ci = 1 / math.sqrt(m)
            else:
                ci = math.sqrt(2) / math.sqrt(m)
            if (col == 0):
                cj = 1 / math.sqrt(n)
            else:
                cj = math.sqrt(2) / math.sqrt(n)
            
            sum = 0
            for k in range(m):
                for l in range (n):
                    dct1 = matrix[k][l] * math.cos((2 * k + 1) * ligne * math.pi / (2 * m)) * math.cos((2 * l + 1) * col * math.pi / (2 * n))
                    sum = sum + dct1
            
            dct[ligne][col] = ci * cj * sum
            
    return dct

def reverseDctTransform(matrix):
    pass

def dct2(a):
    #https://inst.eecs.berkeley.edu/~ee123/sp16/Sections/JPEG_DCT_Demo.html
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    #https://inst.eecs.berkeley.edu/~ee123/sp16/Sections/JPEG_DCT_Demo.html
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def dct(YBlocks, CbBlocks, CrBlocks):
    YBlocks = dct2(YBlocks)
    CbBlocks = dct2(CbBlocks)
    CrBlocks = dct2(CrBlocks)
    return YBlocks, CbBlocks, CrBlocks

def idct(YBlocks, CbBlocks, CrBlocks):
    YBlocks = idct2(YBlocks)
    CbBlocks = idct2(CbBlocks)
    CrBlocks = idct2(CrBlocks)
    return YBlocks, CbBlocks, CrBlocks
    

quantizationBlock =[  [16,  11,  10,  16,  24,  40,  51,  61],
  [12,  12,  14,  19,  26,  58,  60,  55],
  [14,  13,  16,  24,  40,  57,  69,  56],
  [14,  17,  22,  29,  51,  87,  80,  62],
  [18,  22,  37,  56,  68, 109, 103,  77],
  [24,  35,  55,  64,  81, 104, 113,  92],
  [49,  64,  78,  87, 103, 121, 120, 101],
  [72,  92,  95,  98, 112, 100, 103,  99]]

def quantizeBlocks(blocks):
    for k in range(len(blocks)):
        block = blocks[k]
        for i in range(8):
            for j in range(8):
                block[i][j] = round(block[i][j] / quantizationBlock[i][j]);
    return blocks

def dequantizeBlocks(blocks):
    for k in range(len(blocks)):
        block = blocks[k]
        for i in range(8):
            for j in range(8):
                block[i][j] = block[i][j] * quantizationBlock[i][j];
    return blocks


In [11]:
# Vient du github du prof avec quelques modifications 
def huffman(Message):
    if len(Message) > 0:
        #Liste qui sera modifié jusqu'à ce qu'elle contienne seulement la racine de l'arbre
        ArbreSymb =[[Message[0], (Message == Message[0]).sum(), Node(Message[0])]] 
        #dictionnaire obtenu à partir de l'arbre.
        dictionnaire = [[Message[0], '']]
        nbsymboles = 1

        #Recherche des feuilles de l'arbre
        for i in range(1,len(Message)):
            if not list(filter(lambda x: x[0] == Message[i], ArbreSymb)):
                ArbreSymb += [[Message[i], (Message == (Message[i])).sum(),Node(Message[i])]]
                dictionnaire += [[Message[i], '']]
                nbsymboles += 1

        ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])
        #print(ArbreSymb)

        while len(ArbreSymb) > 1:
            #Fusion des noeuds de poids plus faibles
            symbfusionnes = ArbreSymb[0][0] + ArbreSymb[1][0] 
            #Création d'un nouveau noeud
            noeud = Node(symbfusionnes)
            temp = [symbfusionnes, ArbreSymb[0][1] + ArbreSymb[1][1], noeud]
            #Ajustement de l'arbre pour connecter le nouveau avec ses parents 
            ArbreSymb[0][2].parent = noeud
            ArbreSymb[1][2].parent = noeud
            #Enlève les noeuds fusionnés de la liste de noeud à fusionner.
            del ArbreSymb[0:2]
            #Ajout du nouveau noeud à la liste et tri.
            ArbreSymb += [temp]

        ArbreCodes = Node('')
        noeud = ArbreCodes
        parcoursprefix = [node for node in PreOrderIter(ArbreSymb[0][2])]
        parcoursprefix = parcoursprefix[1:len(parcoursprefix)] #ignore la racine

        Prevdepth = 0 #pour suivre les mouvements en profondeur dans l'arbre
        for node in parcoursprefix:  #Liste des noeuds 
            if Prevdepth < node.depth: #On va plus profond dans l'arbre, on met un 0
                temp = Node(noeud.name + '0')
                noeud.children = [temp]
                if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
                    noeud = temp
            elif Prevdepth == node.depth: #Même profondeur, autre feuille, on met un 1
                temp = Node(noeud.name + '1')
                noeud.children = [noeud.children[0], temp]  #Ajoute le deuxième enfant
                if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
                    noeud = temp
            else:
                for i in range(Prevdepth-node.depth): #On prend une autre branche, donc on met un 1
                    noeud = noeud.parent #On remontre dans l'arbre pour prendre la prochaine branche non explorée.
                temp = Node(noeud.name + '1')
                noeud.children = [noeud.children[0], temp]
                if node.children:
                    noeud = temp

            Prevdepth = node.depth    

        #print('\nArbre des codes:\n\n',RenderTree(ArbreCodes, style=AsciiStyle()).by_attr())         
        #print('\nArbre des symboles:\n\n', RenderTree(ArbreSymb[0][2], style=AsciiStyle()).by_attr())  

        ArbreSymbList = [node for node in PreOrderIter(ArbreSymb[0][2])]
        ArbreCodeList = [node for node in PreOrderIter(ArbreCodes)]

        for i in range(len(ArbreSymbList)):
            if ArbreSymbList[i].is_leaf: #Génère des codes pour les feuilles seulement
                temp = list(filter(lambda x: x[0] == ArbreSymbList[i].name, dictionnaire))
                if temp:
                    indice = dictionnaire.index(temp[0])
                    dictionnaire[indice][1] = ArbreCodeList[i].name

        return [dictionnaire]
    else:
        return [[[]]]

def huffmanForAll(blocks):
    block = np.ndarray.flatten(blocks[0,:])
    code = huffman(block[block != 0].astype(int)) 
    for i in range(1, len(blocks) - 1):
        block = np.ndarray.flatten(blocks[i,:])
        code += huffman(block[block != 0].astype(int))
    return code
    

In [21]:
def findHuffman(value, huffmanCode):
    for i in range(len(huffmanCode)):
        if huffmanCode[i][0] == value:
            return len(huffmanCode[i][1]), huffmanCode[i][1]
        
    
def RLE(block, huffmanCode):
    Code = [[0, 0, 0]]
    zeroCount = 0
    for i in range(len(block)):
        for j in range(8):
            if block[i][j] == 0:
                zeroCount += 1
            else:
                numBits, huffman = findHuffman(block[i][j], huffmanCode)
                Code += [[zeroCount, numBits, huffman]]
                zeroCount = 0
    Code.pop(0)
    return Code

def RLEforAll(blocks, huffmanCodes):
    CodeForAll = [RLE(blocks[0], huffmanCodes[0])]
    for i in range(1, len(blocks) -1):
        CodeForAll += [RLE(blocks[i], huffmanCodes[i])]
    return CodeForAll

In [85]:
def decodeHuffman(dictionnary, value):
    for i in range(len(dictionnary)):
        if dictionnary[i][1] == value:
            return dictionnary[i][0]
            
        
def decodeRLE(dictionnary, message):
    decodedMessage = []
    for i in range(len(message)):
        decodedLine = []
        for j in range(len(message[i])):
            if message[i][j][0] != 0:
                for k in range(message[i][j][0]):
                    decodedLine.append(0)
            decodedLine.append(decodeHuffman(dictionnary[i], message[i][j][2]))
        for m in range(64 - len(decodedLine)):
            decodedLine.append(0)
        decodedMessage += decodedLine
    return np.reshape(decodedMessage, (int(len(decodedMessage)/64), 8, 8))
                

In [104]:
y, cb, cr = encode(ycbcr)

YBlocks, CbBlocks, CrBlocks = divide8(y, cb, cr)

YBlocks, CbBlocks, CrBlocks = dct(YBlocks, CbBlocks, CrBlocks)

YBlocks = quantizeBlocks(YBlocks)
CbBlocks = quantizeBlocks(CbBlocks)
CrBlocks = quantizeBlocks(CrBlocks)


# Encode huffman
huffmanYBlocks = huffmanForAll(YBlocks)
huffmanCbBlocks = huffmanForAll(CbBlocks)
huffmanCrBlocks = huffmanForAll(CrBlocks)

# Encode RLE
EncodedYBlocks = RLEforAll(YBlocks, huffmanYBlocks)
EncodedCbBlocks = RLEforAll(CbBlocks, huffmanCbBlocks)
EncodedCrBlocks = RLEforAll(CrBlocks, huffmanCrBlocks)


# DECOMPRESSION #

DecodedYBlocks = decodeRLE(huffmanYBlocks, EncodedYBlocks)
DecodedCbBlocks = decodeRLE(huffmanCbBlocks, EncodedCbBlocks)
DecodedCrBlocks = decodeRLE(huffmanCrBlocks, EncodedCrBlocks)

YBlocks = dequantizeBlocks(DecodedYBlocks)
CbBlocks = dequantizeBlocks(DecodedCbBlocks)
CrBlocks = dequantizeBlocks(DecodedCrBlocks)


YBlocks, CbBlocks, CrBlocks = idct(YBlocks, CbBlocks, CrBlocks)

y, cb, cr = undivide8(YBlocks, CbBlocks, CrBlocks, y.shape[1], y.shape[0])


image = decode(y, cb, cr)